In [253]:
import pandas as pd
import numpy as np
import re, pytz, os, requests, sys
from pathlib import Path
from datetime import datetime
import sys
sys.path.append("/workspaces/service-data")

from src.clean import clean_percentage, clean_fiscal_yr, normalize_string, standardize_column_names
from src.load import load_csv
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

base_dir = Path.cwd()
parent_dir = base_dir.parent

In [254]:
si = pd.read_csv("https://github.com/gcperformance/service-data/releases/latest/download/si.csv", 
                 keep_default_na=False, 
                 na_values='', 
                 delimiter=';',
                 engine='python',
                 skipfooter=2
                 )

snap_si = pd.read_csv("https://github.com/gcperformance/service-data/releases/latest/download/2025-03-01_si.csv", 
                 keep_default_na=False, 
                 na_values='', 
                 delimiter=';',
                 engine='python',
                 skipfooter=2
                 )

key = 'fy_org_id_service_id'

snap_si.set_index(key, inplace=True)
si.set_index(key, inplace=True)

df_base = si.copy()
df_comp = snap_si.copy()


In [256]:
diff_wide = df_base.fillna(0).ne(df_comp.fillna(0)).reset_index()
diff = diff_wide.melt(value_vars = diff_wide.columns, id_vars = key)
diff = diff[diff['value']]

diff.set_index([key, 'variable'], inplace=True)

diff

value
fy_org_id_service_id variable                                                 
2023-2024_223_1728   automated_decision_system                            True
                     automated_decision_system_description_en             True
                     automated_decision_system_description_fr             True
                     client_feedback_channel                              True
                     client_target_groups                                 True
                     cra_bn_identifier_usage                              True
                     department_en                                        True
                     department_fr                                        True
                     fiscal_yr                                            True
                     how_has_the_service_been_assessed_for_accessibi...   True
                     ident_platform                                       True
                     ident_platform_comments                              True
                     last_GBA                                             True
                     last_service_improvement                             True
                     last_service_review                                  True
                     num_applications_by_email                            True
                     num_applications_by_fax                              True
                     num_applications_by_mail                             True
                     num_applications_by_other                            True
                     num_applications_by_phone                            True
                     num_applications_in_person                           True
                     num_applications_online                              True
                     num_applications_total                               True
                     num_phone_enquiries                                  True
                     num_website_visits                                   True
                     org_id                                               True
                     org_name_variant                                     True
                     os_account_registration                              True
                     os_application                                       True
                     os_authentication                                    True
                     os_comments_client_interaction_en                    True
                     os_comments_client_interaction_fr                    True
                     os_decision                                          True
                     os_issuance                                          True
                     os_issue_resolution_feedback                         True
                     program_id                                           True
                     program_name_en                                      True
                     program_name_fr                                      True
                     service_description_en                               True
                     service_description_fr                               True
                     service_fee                                          True
                     service_id                                           True
                     service_name_en                                      True
                     service_name_fr                                      True
                     service_recipient_type                               True
                     service_scope                                        True
                     service_type                                         True
                     service_uri_en                                       True
                     service_uri_fr                                       True
                     sin_usage              

In [257]:

df_base = df_base.reset_index().melt(value_vars = df_base.columns, id_vars=[key])
df_base.set_index([key, 'variable'], inplace=True)

df_comp = df_comp.reset_index().melt(value_vars = df_comp.columns, id_vars=[key])
df_comp.set_index([key, 'variable'], inplace=True)

In [262]:
result = diff.join(df_base, how='left', rsuffix='_base').join(df_comp, how='left', rsuffix='_comp')
result.drop(columns=['value'], inplace=True)
result.reset_index()

,fy_org_id_service_id,variable,value_base,value_comp
0,2023-2024_223_1728,automated_decision_system,NaN,N
1,2023-2024_223_1728,automated_decision_system_description_en,NaN,NaN
2,2023-2024_223_1728,automated_decision_system_description_fr,NaN,NaN
3,2023-2024_223_1728,client_feedback_channel,NaN,"EML, ONL, TEL"
4,2023-2024_223_1728,client_target_groups,NaN,"ECONOM, NGO, PERSON"
5,2023-2024_223_1728,cra_bn_identifier_usage,NaN,N
6,2023-2024_223_1728,department_en,NaN,National Film Board
7,2023-2024_223_1728,department_fr,NaN,Office national du film
8,2023-2024_223_1728,fiscal_yr,NaN,2023-2024
9,2023-2024_223_1728,how_has_the_service_been_assessed_for_accessib...,NaN,NaN
